El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [39]:
import pandas as pd
import numpy as np



Procedemos a leer los datos de la base de datos usando la librería pandas, una vez importados los datos, procederemos a ver los datos, en que condicion estan, si faltan valores y en que cantidad. Tambien si los tipos de datos corresponden a la columna.

In [40]:
df=pd.read_csv('car_data.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [41]:

print("Conteo de valores nulos por columna:")
missing_data = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_info = pd.DataFrame({'Missing Count': missing_data,'Percentage': missing_percentage}).sort_values('Percentage', ascending=False)
print(missing_info)

Conteo de valores nulos por columna:
                   Missing Count  Percentage
NotRepaired                71154   20.079070
VehicleType                37490   10.579368
FuelType                   32895    9.282697
Gearbox                    19833    5.596709
Model                      19705    5.560588
Price                          0    0.000000
RegistrationYear               0    0.000000
DateCrawled                    0    0.000000
Mileage                        0    0.000000
Power                          0    0.000000
RegistrationMonth              0    0.000000
Brand                          0    0.000000
DateCreated                    0    0.000000
NumberOfPictures               0    0.000000
PostalCode                     0    0.000000
LastSeen                       0    0.000000


In [42]:
df[['NotRepaired','VehicleType','FuelType','Gearbox','Model']] = df[['NotRepaired','VehicleType','FuelType','Gearbox','Model']].fillna('unknown')
df[['VehicleType','Gearbox','FuelType','NotRepaired']]=df[['VehicleType','Gearbox','FuelType','NotRepaired']].astype('category')


Voy a comenzar a tratar los valores ausentes. en el caso de "NotRepaired" como puede ser si o no, y tenemos este valor pero tiene un alto porcentaje de mi dataset, pondre otro valor como "unknow".

In [43]:
print("Conteo de valores nulos por columna:")
missing_data = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_info = pd.DataFrame({'Missing Count': missing_data,'Percentage': missing_percentage}).sort_values('Percentage', ascending=False)
print(missing_info)


Conteo de valores nulos por columna:
                   Missing Count  Percentage
DateCrawled                    0         0.0
Price                          0         0.0
VehicleType                    0         0.0
RegistrationYear               0         0.0
Gearbox                        0         0.0
Power                          0         0.0
Model                          0         0.0
Mileage                        0         0.0
RegistrationMonth              0         0.0
FuelType                       0         0.0
Brand                          0         0.0
NotRepaired                    0         0.0
DateCreated                    0         0.0
NumberOfPictures               0         0.0
PostalCode                     0         0.0
LastSeen                       0         0.0


Terminamos de ver el tema de valores ausentes , como las columnas eran categoricas por eso se procedio a imputar en todas la palabra "Unknown". Ahora procederemos a ver la coherencia de los datos.

In [44]:
print(df.describe())

               Price  RegistrationYear          Power        Mileage  \
count  354369.000000     354369.000000  354369.000000  354369.000000   
mean     4416.656776       2004.234448     110.094337  128211.172535   
std      4514.158514         90.227958     189.850405   37905.341530   
min         0.000000       1000.000000       0.000000    5000.000000   
25%      1050.000000       1999.000000      69.000000  125000.000000   
50%      2700.000000       2003.000000     105.000000  150000.000000   
75%      6400.000000       2008.000000     143.000000  150000.000000   
max     20000.000000       9999.000000   20000.000000  150000.000000   

       RegistrationMonth  NumberOfPictures     PostalCode  
count      354369.000000          354369.0  354369.000000  
mean            5.714645               0.0   50508.689087  
std             3.726421               0.0   25783.096248  
min             0.000000               0.0    1067.000000  
25%             3.000000               0.0   30165.

Observamos varias anomalais en los datos, como que en precio hhubo una venta por 0 euros, en el caso del año del registro hay desde 1000 hasta 9999, en el caso de power hay un valor de 20000 y en el kilometraje hay un valor de 9999999, todos estos valores no son coherentes con la realidad, por lo que se procederá a eliminarlos.

In [45]:
df['Price']=df['Price'].drop(df[df['Price']<=100].index)
df['Power']=df['Power'].drop(df[(df['Power']>=10000) | (df['Power']<10)].index)
df['RegistrationYear']=df['RegistrationYear'].drop(df[(df['RegistrationYear']<1900) | (df['RegistrationYear']>2025)].index)
print(df.describe())



               Price  RegistrationYear          Power        Mileage  \
count  340024.000000     354198.000000  313879.000000  354369.000000   
mean     4602.471902       2003.084789     122.330895  128211.172535   
std      4514.902742          7.536418     106.232815   37905.341530   
min       101.000000       1910.000000      10.000000    5000.000000   
25%      1200.000000       1999.000000      75.000000  125000.000000   
50%      2900.000000       2003.000000     110.000000  150000.000000   
75%      6500.000000       2008.000000     150.000000  150000.000000   
max     20000.000000       2019.000000    9710.000000  150000.000000   

       RegistrationMonth  NumberOfPictures     PostalCode  
count      354369.000000          354369.0  354369.000000  
mean            5.714645               0.0   50508.689087  
std             3.726421               0.0   25783.096248  
min             0.000000               0.0    1067.000000  
25%             3.000000               0.0   30165.

Listo tenemos el df limpio y listo para entrenar el modelo.

## Entrenamiento del modelo 

## Análisis del modelo

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos